In [15]:
#pip install -q pyomo
#pip install gurobipy

# Solution 3a
import pyomo.environ as pyo

# sets (all caps)
CUTTING_PATTERNS = ['1','2', '3', '4', '5', '6']

WIDTHS = ['20', '45', '50', '55', '75']

# parameters ((descriptive names in lowercase letters))
orders = {'20': 48, '45': 35, '50' : 24, '55' : 10, '75' : 8}

number_widths_in_pattern = {('1', '20'): 3, ('1', '45'): 0, ('1', '50'): 1, ('1', '55'): 0, ('1', '75'): 0,
                           ('2', '20'): 1, ('2', '45'): 2, ('2', '50'): 0, ('2', '55'): 0, ('2', '75'): 0,
                           ('3', '20'): 0, ('3', '45'): 0, ('3', '50'): 1, ('3', '55'): 1, ('3', '75'): 0,
                           ('4', '20'): 2, ('4', '45'): 0, ('4', '50'): 0, ('4', '55'): 1, ('4', '75'): 0,
                           ('5', '20'): 1, ('5', '45'): 0, ('5', '50'): 0, ('5', '55'): 0, ('5', '75'): 1,
                           ('6', '20'): 3, ('6', '45'): 1, ('6', '50'): 0, ('6', '55'): 0, ('6', '75'): 0}

# Model
model = pyo.ConcreteModel()

# Bounds
model.x = pyo.Var(CUTTING_PATTERNS, domain=pyo.NonNegativeReals)

model.y = pyo.Var(WIDTHS, domain=pyo.NonNegativeReals)


#Objective
def obj_rule(model):
    return sum(model.x[i] for i in CUTTING_PATTERNS)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

#Constraints
def enough_cuts(model, j):
    return (sum(number_widths_in_pattern[i, j]*model.x[i] for i in CUTTING_PATTERNS) >= orders[j])

model.enough_cuts_const = pyo.Constraint(WIDTHS, rule=enough_cuts)


#Solve
result = pyo.SolverFactory("gurobi").solve(model, tee=True)

#Optimal?
print(f'The solver returned a status of: {result.solver.termination_condition}')

#Optimal solution
if result.solver.termination_condition == pyo.TerminationCondition.optimal:
    print(f"Optimal rolls used: {pyo.value(model.obj)}")
    print("Optimal solution:")
    
    for i in CUTTING_PATTERNS:
        print(f"  cutting pattern {i}  = {pyo.value(model.x[i])}")

Restricted license - for non-production use only - expires 2024-10-28
Read LP format model from file C:\Users\Andre\AppData\Local\Temp\tmpozms91hp.pyomo.lp
Reading time = 0.00 seconds
x1: 5 rows, 6 columns, 12 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 9 4900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x84d88620
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 5e+01]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 4 rows, 5 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+00   6.150000e+01   0.000000e+00      0s
       3    4.9500000e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterati

In [10]:
#pip install -q pyomo
#pip install gurobipy

# Solution 3b
import pyomo.environ as pyo

# sets (all caps)
CUTTING_PATTERNS = ['1','2', '3', '4', '5', '6']

WIDTHS = ['20', '45', '50', '55', '75']

# parameters ((descriptive names in lowercase letters))
orders = {'20': 48, '45': 35, '50' : 24, '55' : 10, '75' : 8}

number_widths_in_pattern = {('1', '20'): 3, ('1', '45'): 0, ('1', '50'): 1, ('1', '55'): 0, ('1', '75'): 0,
                           ('2', '20'): 1, ('2', '45'): 2, ('2', '50'): 0, ('2', '55'): 0, ('2', '75'): 0,
                           ('3', '20'): 0, ('3', '45'): 0, ('3', '50'): 1, ('3', '55'): 1, ('3', '75'): 0,
                           ('4', '20'): 2, ('4', '45'): 0, ('4', '50'): 0, ('4', '55'): 1, ('4', '75'): 0,
                           ('5', '20'): 1, ('5', '45'): 0, ('5', '50'): 0, ('5', '55'): 0, ('5', '75'): 1,
                           ('6', '20'): 3, ('6', '45'): 1, ('6', '50'): 0, ('6', '55'): 0, ('6', '75'): 0}

# Model
model = pyo.ConcreteModel()

# Bounds
model.x = pyo.Var(CUTTING_PATTERNS, domain=pyo.NonNegativeReals)

model.y = pyo.Var(WIDTHS, domain=pyo.NonNegativeReals)


#Objective
def obj_rule(model):
    return sum(model.x[i] for i in CUTTING_PATTERNS)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

#Constraints
def enough_cuts_min(model, j):
    return (sum(number_widths_in_pattern[i, j]*model.x[i] for i in CUTTING_PATTERNS) >= orders[j] * 0.9)

model.enough_cuts_min_const = pyo.Constraint(WIDTHS, rule=enough_cuts_min)

def enough_cuts_max(model, k):
    return (sum(number_widths_in_pattern[i, k]*model.x[i] for i in CUTTING_PATTERNS) <= orders[k] * 1.4)

model.enough_cuts_max_const = pyo.Constraint(WIDTHS, rule=enough_cuts_max)


#Solve
result = pyo.SolverFactory("gurobi").solve(model, tee=True)

#Optimal?
print(f'The solver returned a status of: {result.solver.termination_condition}')

#Optimal solution
if result.solver.termination_condition == pyo.TerminationCondition.optimal:
    print(f"Optimal rolls used: {pyo.value(model.obj)}")
    print("Optimal solution:")
    
    for i in CUTTING_PATTERNS:
        print(f"  cutting pattern {i} = {pyo.value(model.x[i])}")

Restricted license - for non-production use only - expires 2024-10-28
Read LP format model from file C:\Users\Andre\AppData\Local\Temp\tmp7an55qt4.pyomo.lp
Reading time = 0.00 seconds
x1: 10 rows, 6 columns, 24 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 9 4900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 6 columns and 24 nonzeros
Model fingerprint: 0xf9526194
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+00, 7e+01]
Presolve removed 6 rows and 0 columns
Presolve time: 0.00s
Presolved: 4 rows, 10 columns, 15 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.2000000e+00   1.608750e+01   0.000000e+00      0s
       3    4.4550000e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iter

In [14]:
#pip install -q pyomo
#pip install gurobipy

# Solution 3c
import pyomo.environ as pyo

# sets (all caps)
CUTTING_PATTERNS = ['1','2', '3', '4', '5', '6', '7']

WIDTHS = ['20', '45', '50', '55', '75']

# parameters ((descriptive names in lowercase letters))
orders = {'20': 48, '45': 35, '50' : 24, '55' : 10, '75' : 8}

number_widths_in_pattern = {('1', '20'): 3, ('1', '45'): 0, ('1', '50'): 1, ('1', '55'): 0, ('1', '75'): 0,
                           ('2', '20'): 1, ('2', '45'): 2, ('2', '50'): 0, ('2', '55'): 0, ('2', '75'): 0,
                           ('3', '20'): 0, ('3', '45'): 0, ('3', '50'): 1, ('3', '55'): 1, ('3', '75'): 0,
                           ('4', '20'): 2, ('4', '45'): 0, ('4', '50'): 0, ('4', '55'): 1, ('4', '75'): 0,
                           ('5', '20'): 1, ('5', '45'): 0, ('5', '50'): 0, ('5', '55'): 0, ('5', '75'): 1,
                           ('6', '20'): 3, ('6', '45'): 1, ('6', '50'): 0, ('6', '55'): 0, ('6', '75'): 0,
                           ('7', '20'): 0, ('7', '45'): 0, ('7', '50'): 2, ('7', '55'): 0, ('7', '75'): 0}

# Model
model = pyo.ConcreteModel()

# Bounds
model.x = pyo.Var(CUTTING_PATTERNS, domain=pyo.NonNegativeReals)

model.y = pyo.Var(WIDTHS, domain=pyo.NonNegativeReals)


#Objective
def obj_rule(model):
    return sum(model.x[i] for i in CUTTING_PATTERNS)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

#Constraints
def enough_cuts(model, j):
    return (sum(number_widths_in_pattern[i, j]*model.x[i] for i in CUTTING_PATTERNS) >= orders[j])

model.enough_cuts_const = pyo.Constraint(WIDTHS, rule=enough_cuts)


#Solve
result = pyo.SolverFactory("gurobi").solve(model, tee=True)

#Optimal?
print(f'The solver returned a status of: {result.solver.termination_condition}')

#Optimal solution
if result.solver.termination_condition == pyo.TerminationCondition.optimal:
    print(f"Optimal rolls used: {pyo.value(model.obj)}")
    print("Optimal solution:")
    
    for i in CUTTING_PATTERNS:
        print(f"  cutting pattern {i} = {pyo.value(model.x[i])}")

Restricted license - for non-production use only - expires 2024-10-28
Read LP format model from file C:\Users\Andre\AppData\Local\Temp\tmp4dsg3u7i.pyomo.lp
Reading time = 0.00 seconds
x1: 5 rows, 7 columns, 13 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 9 4900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 7 columns and 13 nonzeros
Model fingerprint: 0x26b2488d
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 5e+01]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 4 rows, 6 columns, 11 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+00   4.950000e+01   0.000000e+00      0s
       4    4.6250000e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterati

In [2]:
#pip install -q pyomo
#pip install gurobipy

# Solution 3da
import pyomo.environ as pyo

# sets (all caps)
CUTTING_PATTERNS = ['1','2', '3', '4', '5', '6']

WIDTHS = ['20', '45', '50', '55', '75']

# parameters ((descriptive names in lowercase letters))
orders = {'20': 48, '45': 35, '50' : 24, '55' : 10, '75' : 8}

number_widths_in_pattern = {('1', '20'): 3, ('1', '45'): 0, ('1', '50'): 1, ('1', '55'): 0, ('1', '75'): 0,
                           ('2', '20'): 1, ('2', '45'): 2, ('2', '50'): 0, ('2', '55'): 0, ('2', '75'): 0,
                           ('3', '20'): 0, ('3', '45'): 0, ('3', '50'): 1, ('3', '55'): 1, ('3', '75'): 0,
                           ('4', '20'): 2, ('4', '45'): 0, ('4', '50'): 0, ('4', '55'): 1, ('4', '75'): 0,
                           ('5', '20'): 1, ('5', '45'): 0, ('5', '50'): 0, ('5', '55'): 0, ('5', '75'): 1,
                           ('6', '20'): 3, ('6', '45'): 1, ('6', '50'): 0, ('6', '55'): 0, ('6', '75'): 0}

# Model
model = pyo.ConcreteModel()

# Bounds
model.x = pyo.Var(CUTTING_PATTERNS, domain=pyo.NonNegativeIntegers)

model.y = pyo.Var(WIDTHS, domain=pyo.NonNegativeIntegers)


#Objective
def obj_rule(model):
    return sum(model.x[i] for i in CUTTING_PATTERNS)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

#Constraints
def enough_cuts(model, j):
    return (sum(number_widths_in_pattern[i, j]*model.x[i] for i in CUTTING_PATTERNS) >= orders[j])

model.enough_cuts_const = pyo.Constraint(WIDTHS, rule=enough_cuts)


#Solve
result = pyo.SolverFactory("gurobi").solve(model, tee=True)

#Optimal?
print(f'The solver returned a status of: {result.solver.termination_condition}')

#Optimal solution
if result.solver.termination_condition == pyo.TerminationCondition.optimal:
    print(f"Optimal rolls used: {pyo.value(model.obj)}")
    print("Optimal solution:")
    
    for i in CUTTING_PATTERNS:
        print(f"  cutting pattern {i}  = {pyo.value(model.x[i])}")

Restricted license - for non-production use only - expires 2024-10-28
Read LP format model from file C:\Users\Andre\AppData\Local\Temp\tmpa_dtiqpu.pyomo.lp
Reading time = 0.00 seconds
x1: 5 rows, 6 columns, 12 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 9 4900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xf1ba283d
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 5e+01]
Found heuristic solution: objective 67.0000000
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 4 rows, 5 columns, 10 nonzeros
Variable types: 0 continuous, 5 integer (0 binary)

Root relaxation: objective 4.950000e+01, 5 iterations, 0.00 sec

In [11]:
#pip install -q pyomo
#pip install gurobipy

# Solution 3b
import pyomo.environ as pyo

# sets (all caps)
CUTTING_PATTERNS = ['1','2', '3', '4', '5', '6']

WIDTHS = ['20', '45', '50', '55', '75']

# parameters ((descriptive names in lowercase letters))
orders = {'20': 48, '45': 35, '50' : 24, '55' : 10, '75' : 8}

number_widths_in_pattern = {('1', '20'): 3, ('1', '45'): 0, ('1', '50'): 1, ('1', '55'): 0, ('1', '75'): 0,
                           ('2', '20'): 1, ('2', '45'): 2, ('2', '50'): 0, ('2', '55'): 0, ('2', '75'): 0,
                           ('3', '20'): 0, ('3', '45'): 0, ('3', '50'): 1, ('3', '55'): 1, ('3', '75'): 0,
                           ('4', '20'): 2, ('4', '45'): 0, ('4', '50'): 0, ('4', '55'): 1, ('4', '75'): 0,
                           ('5', '20'): 1, ('5', '45'): 0, ('5', '50'): 0, ('5', '55'): 0, ('5', '75'): 1,
                           ('6', '20'): 3, ('6', '45'): 1, ('6', '50'): 0, ('6', '55'): 0, ('6', '75'): 0}

# Model
model = pyo.ConcreteModel()

# Bounds
model.x = pyo.Var(CUTTING_PATTERNS, domain=pyo.NonNegativeIntegers)

model.y = pyo.Var(WIDTHS, domain=pyo.NonNegativeIntegers)


#Objective
def obj_rule(model):
    return sum(model.x[i] for i in CUTTING_PATTERNS)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

#Constraints
def enough_cuts_min(model, j):
    return (sum(number_widths_in_pattern[i, j]*model.x[i] for i in CUTTING_PATTERNS) >= orders[j] * 0.9)

model.enough_cuts_min_const = pyo.Constraint(WIDTHS, rule=enough_cuts_min)

def enough_cuts_max(model, k):
    return (sum(number_widths_in_pattern[i, k]*model.x[i] for i in CUTTING_PATTERNS) <= orders[k] * 1.4)

model.enough_cuts_max_const = pyo.Constraint(WIDTHS, rule=enough_cuts_max)


#Solve
result = pyo.SolverFactory("gurobi").solve(model, tee=True)

#Optimal?
print(f'The solver returned a status of: {result.solver.termination_condition}')

#Optimal solution
if result.solver.termination_condition == pyo.TerminationCondition.optimal:
    print(f"Optimal rolls used: {pyo.value(model.obj)}")
    print("Optimal solution:")
    
    for i in CUTTING_PATTERNS:
        print(f"  cutting pattern {i} = {pyo.value(model.x[i])}")

Restricted license - for non-production use only - expires 2024-10-28
Read LP format model from file C:\Users\Andre\AppData\Local\Temp\tmpdapa0ezz.pyomo.lp
Reading time = 0.00 seconds
x1: 10 rows, 6 columns, 24 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 9 4900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 6 columns and 24 nonzeros
Model fingerprint: 0x65f7f3b0
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+00, 7e+01]
Presolve removed 10 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 46 

Optimal solutio

In [9]:
#pip install -q pyomo
#pip install gurobipy

# Solution 3c
import pyomo.environ as pyo

# sets (all caps)
CUTTING_PATTERNS = ['1','2', '3', '4', '5', '6', '7']

WIDTHS = ['20', '45', '50', '55', '75']

# parameters ((descriptive names in lowercase letters))
orders = {'20': 48, '45': 35, '50' : 24, '55' : 10, '75' : 8}

number_widths_in_pattern = {('1', '20'): 3, ('1', '45'): 0, ('1', '50'): 1, ('1', '55'): 0, ('1', '75'): 0,
                           ('2', '20'): 1, ('2', '45'): 2, ('2', '50'): 0, ('2', '55'): 0, ('2', '75'): 0,
                           ('3', '20'): 0, ('3', '45'): 0, ('3', '50'): 1, ('3', '55'): 1, ('3', '75'): 0,
                           ('4', '20'): 2, ('4', '45'): 0, ('4', '50'): 0, ('4', '55'): 1, ('4', '75'): 0,
                           ('5', '20'): 1, ('5', '45'): 0, ('5', '50'): 0, ('5', '55'): 0, ('5', '75'): 1,
                           ('6', '20'): 3, ('6', '45'): 1, ('6', '50'): 0, ('6', '55'): 0, ('6', '75'): 0,
                           ('7', '20'): 0, ('7', '45'): 0, ('7', '50'): 2, ('7', '55'): 0, ('7', '75'): 0}

# Model
model = pyo.ConcreteModel()

# Bounds
model.x = pyo.Var(CUTTING_PATTERNS, domain=pyo.NonNegativeIntegers)

model.y = pyo.Var(WIDTHS, domain=pyo.NonNegativeIntegers)


#Objective
def obj_rule(model):
    return sum(model.x[i] for i in CUTTING_PATTERNS)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

#Constraints
def enough_cuts(model, j):
    return (sum(number_widths_in_pattern[i, j]*model.x[i] for i in CUTTING_PATTERNS) >= orders[j])

model.enough_cuts_const = pyo.Constraint(WIDTHS, rule=enough_cuts)


#Solve
result = pyo.SolverFactory("gurobi").solve(model, tee=True)

#Optimal?
print(f'The solver returned a status of: {result.solver.termination_condition}')

#Optimal solution
if result.solver.termination_condition == pyo.TerminationCondition.optimal:
    print(f"Optimal rolls used: {pyo.value(model.obj)}")
    print("Optimal solution:")
    
    for i in CUTTING_PATTERNS:
        print(f"  cutting pattern {i} = {pyo.value(model.x[i])}")

Restricted license - for non-production use only - expires 2024-10-28
Read LP format model from file C:\Users\Andre\AppData\Local\Temp\tmphiuji2lq.pyomo.lp
Reading time = 0.01 seconds
x1: 5 rows, 7 columns, 13 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 9 4900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 7 columns and 13 nonzeros
Model fingerprint: 0xca069e00
Variable types: 0 continuous, 7 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 5e+01]
Found heuristic solution: objective 65.0000000
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 4 rows, 6 columns, 11 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)

Root relaxation: objective 4.625000e+01, 5 iterations, 0.00 sec